In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon

# Función para simular tiros libres con una probabilidad dada
def simular_tiros(prob, qty):
    return np.sum(np.random.rand(qty) < prob)

# Inicializar el gimnasio con probabilidades de enceste para cada jugadora
def inicializar_gimnasio():
    jugadora_mejor = 0.5
    peloton = np.arange(0.204, 0.400 + 0.002, 0.002)  # 99 jugadoras
    jugadoras = np.append(peloton, jugadora_mejor)
    np.random.shuffle(jugadoras)  # Mezclar para que la mejor no siempre esté al final
    print(f"Inicializado el gimnasio con {len(jugadoras)} jugadoras.")
    return jugadoras

# Ejecuta rondas eliminatorias
def ronda_eliminatoria(jugadoras, tiros, desvio_eliminacion, ronda_numero):
    print(f"\nIniciando ronda {ronda_numero} con {len(jugadoras)} jugadoras activas y {tiros} tiros cada una.")
    encestes = np.array([simular_tiros(prob, tiros) for prob in jugadoras])

    # Calcular el umbral de eliminación
    promedio_encestes = np.mean(encestes)
    umbral_eliminacion = promedio_encestes - desvio_eliminacion * np.std(encestes)

    # Mantener solo jugadoras por encima del umbral
    jugadoras_activas = jugadoras[encestes >= umbral_eliminacion]
    encestes_activas = encestes[encestes >= umbral_eliminacion]
    print(f"Ronda {ronda_numero} - Umbral de eliminación: {umbral_eliminacion:.2f}")
    print(f"Jugadoras que pasan a la siguiente ronda: {len(jugadoras_activas)}")
    return jugadoras_activas, encestes_activas

# Estrategia de selección de la mejor jugadora
def estrategia_seleccion():
    jugadoras = inicializar_gimnasio()
    
    # Configuración de rondas eliminatorias
    rondas_config = [
        {"tiros": 10, "desvio_eliminacion": 0.0},
        {"tiros": 20, "desvio_eliminacion": -0.1},
        {"tiros": 50, "desvio_eliminacion": 0.0},
        {"tiros": 100, "desvio_eliminacion": 0.1}
    ]
    
    # Ejecución de rondas eliminatorias
    for ronda_numero, config in enumerate(rondas_config, start=1):
        tiros = config["tiros"]
        desvio_eliminacion = config["desvio_eliminacion"]
        jugadoras, encestes = ronda_eliminatoria(jugadoras, tiros, desvio_eliminacion, ronda_numero)

        # Si solo queda una jugadora, la seleccionamos como la mejor
        if len(jugadoras) == 1:
            print(f"Seleccionada la jugadora restante en la ronda {ronda_numero}")
            return jugadoras[0] == 0.5  # Verificar si la mejor es la jugadora de probabilidad 0.5

    # Ronda final - Elegir la jugadora con más encestes si quedan varias
    if len(jugadoras) > 1:
        print(f"Iniciando ronda final con {len(jugadoras)} jugadoras.")
        tiros_final = 200
        encestes_final = np.array([simular_tiros(prob, tiros_final) for prob in jugadoras])
        mejor_indice = np.argmax(encestes_final)
        print(f"Jugadora seleccionada en la ronda final con encestes: {encestes_final[mejor_indice]}")
        return jugadoras[mejor_indice] == 0.5  # Verificar si la mejor es la jugadora de probabilidad 0.5

# Estimación Monte Carlo de la tasa de éxito de la estrategia
def monte_carlo_experimentos(n_experimentos=100000):
    aciertos = 0
    for i in range(n_experimentos):
        if i % 10000 == 0:
            print(f"\nCompletados {i} experimentos...")
        aciertos += estrategia_seleccion()
    
    # Cálculo de tasa de éxito
    tasa_aciertos = aciertos / n_experimentos
    print(f"\nTasa de elección correcta: {tasa_aciertos:.4f}")
    return tasa_aciertos

# Ejecutar simulación
if __name__ == "__main__":
    tasa_aciertos = monte_carlo_experimentos()